# Data Creation

- Kevin Chuang
- Yelp Academic Dataset (01.2019)

## Notes

- Already dropped closed businesses in convert_to_csv.ipynb notebook
- Filtering down businesses to only take food-related businesses
- Dropping updated reviews of same user and same business (taking the latest one)
- Filtering out users with less than 10 reviews

In [1]:
import os
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numpy.linalg import norm
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn import base
from ast import literal_eval

%matplotlib inline

## Food businesses

In [2]:
business_df = pd.read_csv("yelp_business.csv")

In [3]:
business_df.shape

(158525, 14)

In [4]:
# Drop businesses with no categories
business_df = business_df.dropna(axis=0, subset=['categories'])

In [5]:
business_df.shape

(158064, 14)

In [6]:
# Let's filter for restaurants and food related businesses

desired_categories = ['Restaurants', 'Food']

re_pattern = "|".join(desired_categories)

food_business_df = business_df[business_df['categories'].str.contains(re_pattern)]

In [7]:
print("Number of food-related businesses: {}".format(food_business_df.shape[0]))

Number of food-related businesses: 54459


In [8]:
food_business_ids = food_business_df['business_id'].values

In [9]:
food_business_df.isnull().values.any()

True

In [10]:
food_business_df.isnull().sum()

address           670
attributes       2447
business_id         0
categories          0
city                0
hours           10519
is_open             0
latitude            0
longitude           0
name                0
postal_code       182
review_count        0
stars               0
state               0
dtype: int64

In [11]:
food_business_df.shape

(54459, 14)

In [12]:
food_business_df = food_business_df.dropna(axis=0, subset=['attributes'])

In [13]:
food_business_df.drop(columns=['address', 'hours', 'is_open', 
                               'latitude', 'longitude', 'review_count'], inplace=True)

In [14]:
food_business_df.rename(columns={'stars': 'avg_business_stars'}, inplace=True)

In [15]:
np.any(food_business_df.duplicated(subset=['business_id']))

False

In [16]:
food_business_df.head()

,attributes,business_id,categories,city,name,postal_code,avg_business_stars,state
0,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,Emerald Chinese Restaurant,L5R 3E7,2.5,ON
1,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,Musashi Japanese Restaurant,28210,4.0,NC
9,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,Taco Bell,85016,3.0,AZ
11,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,Marco's Pizza,44060,4.0,OH
12,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",-K4gAv8_vjx8-2BxkVeRkA,"Bakeries, Food",Willoughby,Baby Cakes,44094,3.0,OH


In [17]:
food_business_df['business_id'].unique().shape

(52012,)

In [18]:
assert food_business_df['business_id'].unique().shape[0] == food_business_df['business_id'].shape[0]

In [ ]:
# food_business_df.to_csv("food_restaurant.csv", index=False)

## Yelp Users

In [19]:
user_df = pd.read_csv("yelp_user.csv")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
user_df.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.03,1,0,1,2,0,0,1,0,1,...,25,"2015,2016,2017",5,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",17,Rashmi,95,84,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33
1,3.63,1,0,1,1,0,0,0,0,0,...,16,NaN,4,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",22,Jenna,33,48,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06
2,3.71,0,0,0,0,0,0,1,0,0,...,10,NaN,0,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",8,David,16,28,bc8C_eETBWL0olvFSJJd0w,2013-10-04 00:16:10
3,4.85,0,0,0,1,0,0,0,0,2,...,14,NaN,5,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",4,Angela,17,30,dD0gZpBctWGdWo9WlGuhlA,2014-05-22 15:57:30
4,4.08,80,0,80,28,1,1,16,5,57,...,665,"2015,2016,2017,2018",39,"mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",279,Nancy,361,1114,MM4RJAeH6yuaN8oZDSt0RA,2013-10-23 07:02:50


In [21]:
user_df.columns

Index(['average_stars', 'compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
       'elite', 'fans', 'friends', 'funny', 'name', 'review_count', 'useful',
       'user_id', 'yelping_since'],
      dtype='object')

In [22]:
user_df.drop(columns=['compliment_cool', 'compliment_cute',
                      'compliment_funny', 'compliment_hot', 'compliment_list',
                      'compliment_more', 'compliment_note', 'compliment_photos',
                     'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
                     'funny', 'name', 'review_count', 'useful', 'friends'], inplace=True)

In [23]:
def convert_elite_to_years(row):
    if pd.isnull(row):
        return 0
    else:
        try:
            x = row.split(",")
        except:
            return 1
        return len(x)

In [24]:
user_df['elite'] = user_df['elite'].map(convert_elite_to_years)

In [25]:
user_df.rename(columns={'average_stars': 'avg_user_star'}, inplace=True)

In [26]:
user_df.head()

,avg_user_star,elite,fans,user_id,yelping_since
0,4.03,3,5,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33
1,3.63,0,4,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06
2,3.71,0,0,bc8C_eETBWL0olvFSJJd0w,2013-10-04 00:16:10
3,4.85,0,5,dD0gZpBctWGdWo9WlGuhlA,2014-05-22 15:57:30
4,4.08,4,39,MM4RJAeH6yuaN8oZDSt0RA,2013-10-23 07:02:50


In [27]:
user_df.isnull().values.any()

False

In [28]:
user_df.shape

(1637138, 5)

## Yelp Reviews

In [29]:
reviews_df = pd.read_csv("yelp_review.csv")

In [30]:
reviews_df.shape

(5823664, 9)

In [31]:
reviews_df.drop(columns=["cool", "funny", "useful"], inplace=True)

In [32]:
reviews_df.isnull().values.any()

True

In [33]:
# Drop null values 
reviews_df.dropna(axis=0, inplace=True)

In [34]:
food_reviews_df = reviews_df[reviews_df['business_id'].isin(food_business_ids)]

In [35]:
food_reviews_df.shape

(3894463, 6)

### Filtering for Reviews dataset

- User ids with no reviews in food businesses
- User ids with less than 10 reviews
- Drop user's past reviews when a user has multiple reviews for a business

In [36]:
food_reviews_df['date'] = pd.to_datetime(food_reviews_df['date'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
np.any(food_reviews_df.duplicated(subset=['user_id', 'business_id'], keep=False))

True

In [38]:
# Only use rows with updated review, drop the rest
food_reviews_df = food_reviews_df.sort_values('date').drop_duplicates(subset=['user_id', 'business_id'], keep='last')

In [39]:
food_reviews_df.shape

(3769910, 6)

In [40]:
food_reviews_df = food_reviews_df[food_reviews_df.groupby("user_id")["user_id"].transform('count').ge(10)]

In [41]:
food_reviews_df.shape

(1693154, 6)

In [42]:
food_reviews_df.head()

,business_id,date,review_id,stars,text,user_id
4651436,u8C8pRvaHXg3PgDrsUHJHQ,2004-10-19 19:24:13,0QHCY_55TFHHvyumEMpDew,4.0,Good stuff. Pricey by normal pizza standards.,nkN_do3fJ9xekchVC-v68A
5239945,EZOoB2D8uQHV_gJoGCMTxQ,2004-10-19 21:33:08,1Iobyi_7BkFON25Oegs0aw,4.0,Love their subs. Cheap and top shelf ingredients.,nkN_do3fJ9xekchVC-v68A
4063729,oYMsq2Xvzw6UbrIlMWjb-A,2004-10-19 21:34:40,2F5J51OYtD49eyIUKJKVgg,4.0,Love their pizza. They used to have a great ta...,nkN_do3fJ9xekchVC-v68A
3475090,ydUqgWsF3F27TbauOyib0w,2004-12-19 20:47:24,Ef1skKLKZ9izwBmreb_-qw,4.0,"Frequently busy due to their great food, but t...",62GNFh5FySkA3MbrQmnqvg
5240922,VMPSdoBgJuyS9t_x_caTig,2005-03-03 19:49:39,39ukJIrbqD1Pk5V16B5weA,5.0,Easily my favorite place to eat in Madison. G...,HLaSqQMDVvlcFPGJL_kGCA


In [43]:
# food_reviews_df.to_csv("food_reviews.csv", index=False)

In [44]:
desired_user_ids = food_reviews_df['user_id'].unique()

In [45]:
print("Number of unique users in reviews: {}".format(desired_user_ids.shape[0]))

Number of unique users in reviews: 64658


In [46]:
user_df = user_df[user_df['user_id'].isin(desired_user_ids)]

In [47]:
user_df.shape

(64658, 5)

In [ ]:
# user_df.to_csv("filtered_users.csv", index=False)

## Merge business and reviews

In [48]:
merged_df = food_reviews_df.merge(food_business_df[['attributes', 'name', 'city','categories', 'business_id', 'avg_business_stars']], 
                                       how = 'inner', on = 'business_id')

In [49]:
merged_df.shape

(1684896, 11)

In [50]:
merged_df = merged_df.dropna()

In [51]:
merged_df.shape

(1684896, 11)

## Merge users to dataset

In [52]:
final_merged_df = merged_df.merge(user_df[['user_id', 'avg_user_star']], 
                                       how = 'inner', on = 'user_id')

In [53]:
final_merged_df.shape

(1684896, 12)

In [54]:
final_merged_df.columns

Index(['business_id', 'date', 'review_id', 'stars', 'text', 'user_id',
       'attributes', 'name', 'city', 'categories', 'avg_business_stars',
       'avg_user_star'],
      dtype='object')

In [55]:
final_merged_df.to_csv("final_yelp_restaurant_data_v3.csv", index=False)